In [1]:
import pandas as pd
import os
import unittest
import string
import re
pd.set_option('display.max_rows', 10)

In [2]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day16_input.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s = file1.read().splitlines()
pd.Series(s)

0                  departure location: 49-239 or 247-960
1                   departure station: 43-135 or 155-963
2                  departure platform: 27-426 or 449-955
3                     departure track: 43-655 or 680-949
4                      departure date: 49-159 or 175-970
                             ...                        
255    938,409,408,854,564,638,742,848,914,885,343,80...
256    350,132,932,390,71,211,58,233,941,904,944,280,...
257    508,561,712,859,495,884,653,934,102,911,703,32...
258    298,541,103,248,694,226,630,796,116,864,887,19...
259    572,367,922,916,416,116,859,589,587,633,688,27...
Length: 260, dtype: object

In [3]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day16_example1.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s_ex1 = file1.read().splitlines()
pd.Series(s_ex1)

0        class: 1-3 or 5-7
1       row: 6-11 or 33-44
2     seat: 13-40 or 45-50
3                         
4             your ticket:
              ...         
7          nearby tickets:
8                   7,3,47
9                  40,4,50
10                 55,2,20
11                 38,6,12
Length: 12, dtype: object

In [4]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day16_example2.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file2 = open(trainFile, 'r') 
s_ex2 = file2.read().splitlines()
pd.Series(s_ex2)

0      class: 0-1 or 4-19
1        row: 0-5 or 8-19
2     seat: 0-13 or 16-19
3                        
4            your ticket:
             ...         
6                        
7         nearby tickets:
8                  3,9,18
9                  15,1,5
10                 5,14,9
Length: 11, dtype: object

In [5]:
def parseField(line):   
    pattern = '(.*): (\\d+)-(\\d+) or (\\d+)-(\\d+)'
    matches = re.match(pattern, line)
    name = matches.group(1)
    l1 = int(matches.group(2))
    l2 = int(matches.group(3))
    r1 = int(matches.group(4))
    r2 = int(matches.group(5))
    return (name, l1, l2, r1, r2)


def parse(lines):
    fields = []
    my_ticket = None
    nearby_tickets = []
    for i in range(0,len(lines)):
        line = lines[i].strip()        
        if line.count(": ") > 0:
            fields.append(parseField(line))
            continue
        parts = line.split(",")
        if line.count(",") > 0:
            ticket = list(map(int,parts))
            if my_ticket == None:
                my_ticket = ticket
            else:
                nearby_tickets.append(ticket)
    return (fields, my_ticket, nearby_tickets)

In [6]:
p = parse(s)
p_ex1 = parse(s_ex1)
p_ex1

([('class', 1, 3, 5, 7), ('row', 6, 11, 33, 44), ('seat', 13, 40, 45, 50)],
 [7, 1, 14],
 [[7, 3, 47], [40, 4, 50], [55, 2, 20], [38, 6, 12]])

In [7]:
def isValid(field, value):
    return (field[1] <= value and value <= field[2]) or (field[3] <= value and value <= field[4])

In [8]:
def sumInvalidFields(fields, ticket):
    result = 0
    isInvalid = False
    for value in ticket:
        found = False
        for f in fields:
            if isValid(f, value):
                found = True
        if not found:
            result += value
            isInvalid = True
    return (result,isInvalid)

In [9]:
def calculatePartOne(input):
    result = 0
    fields = input[0]
    other = input[2]
    for ticket in other:
        result += sumInvalidFields(fields, ticket)[0]
    return result

In [10]:
calculatePartOne(p_ex1)

71

In [11]:
calculatePartOne(p)

22977

In [12]:
def getValidTickets(input):
    result = []
    fields = input[0]
    other = input[2]
    for ticket in other:
        invalid = sumInvalidFields(fields, ticket)[1]
        if not invalid:
            result.append(ticket)
    #result.append(input[1])
    return result

In [13]:
def getPossibleFields(fields, validTickets, i):
    possibilities = []
    for f in range(0,len(fields)):
        field = fields[f]            
        validForAll = True
        for t in validTickets:            
            ticket_value = t[i]
            if not isValid(field, ticket_value):
                validForAll = False
                #print("invalid", field, ticket_value)
                break
            else:
                #print("valid", field, ticket_value)
                pass
        #print("===field===" , field, validForAll)
        if validForAll:
            possibilities.append(f)
    return possibilities

def matchFields(input):
    validTickets = getValidTickets(input)
    fields = input[0]
    my_ticket = input[1]
    result = []
    for i in range(0,len(my_ticket)):
        #print("checking column" ,i)
        possibilities = getPossibleFields(fields, validTickets, i)
        result.append(possibilities)
    return result

In [14]:
p_ex2 = parse(s_ex2)
p_ex2

([('class', 0, 1, 4, 19), ('row', 0, 5, 8, 19), ('seat', 0, 13, 16, 19)],
 [11, 12, 13],
 [[3, 9, 18], [15, 1, 5], [5, 14, 9]])

In [15]:
matchFields(p_ex2)

[[1], [0, 1], [0, 1, 2]]

In [16]:
q = matchFields(p)
q

[[8, 10, 14, 19],
 [3, 7, 8, 10, 14, 18, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 13, 14, 18, 19],
 [7, 8, 10, 14, 18, 19],
 [2, 3, 4, 7, 8, 10, 14, 18, 19],
 [8, 10, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 16, 18, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 18, 19],
 [8, 19],
 [19],
 [0, 2, 3, 4, 7, 8, 10, 14, 18, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 10, 14, 18, 19],
 [2, 3, 7, 8, 10, 14, 18, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 10, 13, 14, 18, 19],
 [0, 2, 3, 4, 5, 7, 8, 10, 14, 18, 19],
 [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 18, 19],
 [8, 10, 14, 18, 19]]

In [17]:
def reduce(input):
    current = input[:]
    for i in range(0,len(input)):
        next = current[:]
        for j in range(0,len(input)):
            choices = current[j]
            if len(choices) == 1:
                singleton = choices[0]
                for k in range(0,len(input)):
                    if k != j and singleton in next[k]:                        
                        next[k].remove(singleton)
        current = next[:]
        next = current[:]
        for j in range(0,len(input)):
            count = 0
            for k in range(0,len(input)):
                if j in current[k]:
                    count += 1
            if count == 1:
                singleton = j
                for k in range(0,len(input)):
                    if singleton in current[k]:
                        next[k] = [singleton]
        current = next[:]
    return current

In [18]:
reduce(q)

[[14],
 [3],
 [11],
 [7],
 [4],
 [10],
 [17],
 [6],
 [12],
 [16],
 [15],
 [8],
 [19],
 [0],
 [1],
 [2],
 [13],
 [5],
 [9],
 [18]]

In [19]:
#help = [14,3,11,7,4,10,17,6,12,16,15,8,19,0,1,2,13,5,9,18]
help = []
for k in reduce(q):
    help.append(k[0])
help

[14, 3, 11, 7, 4, 10, 17, 6, 12, 16, 15, 8, 19, 0, 1, 2, 13, 5, 9, 18]

In [20]:
def solvePartTwo(input):
    result = 1
    my_ticket = input[1]
    for i in range(0,len(help)):
        if help[i] < 6:
            result *= my_ticket[i]
    return result

In [21]:
solvePartTwo(p)

998358379943